In [4]:
import sys
sys.path.append('../')
import pandas as pd
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot

In [5]:
df_raw = pd.read_pickle("../data/GBP_JPY_H1.pkl")

In [7]:
df_raw.shape

(68406, 14)

In [11]:
df_an = df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [12]:
df_an = RSI(df_an)

In [15]:
df_an = apply_patterns(df_an)

In [16]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [17]:
df_an.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'RSI_14',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'EMA_200'],
      dtype='object')

In [ ]:
our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c', 'bid_c', 'direction', 'ENGULFING', 'EMA_200', 'RSI_14']

In [26]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [27]:
df_slim.tail()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,direction,ENGULFING,EMA_200,RSI_14
5796,2025-06-27 19:00:00+00:00,198.193,198.428,198.180,198.412,198.429,198.395,1,False,197.233094,51.435464
5797,2025-06-27 20:00:00+00:00,198.410,198.506,198.344,198.467,198.514,198.420,1,False,197.245371,53.058522
5798,2025-06-29 21:00:00+00:00,197.921,198.294,197.916,198.206,198.431,197.981,1,False,197.254930,45.318336
5799,2025-06-29 22:00:00+00:00,198.167,198.466,198.167,198.298,198.319,198.278,1,False,197.265309,48.187547
5800,2025-06-29 23:00:00+00:00,198.291,198.319,198.188,198.240,198.262,198.218,-1,False,197.275007,46.529953


In [28]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h > row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE
            

In [29]:
df_slim['SIGNAL'] = df_slim.apply(apply_signal, axis=1)

In [30]:
df_slim['SIGNAL'].value_counts()

SIGNAL
 0    5348
 1     284
-1     169
Name: count, dtype: int64

In [31]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5

def apply_take_profit(row):
    if row.SIGNAL != NONE:
        return (row.mid_c - row.mid_o) * PROFIT_FACTOR + row.mid_c
    else:
        return 0.0
def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        return row.mid_o
    else:
        return 0.0

In [32]:
df_slim['TP'] = df_slim.apply(apply_take_profit, axis=1)
df_slim['SL'] = df_slim.apply(apply_stop_loss, axis=1)

In [35]:
df_slim[df_slim.SIGNAL == BUY].head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,direction,ENGULFING,EMA_200,RSI_14,SIGNAL,TP,SL
359,2024-08-13 06:00:00+00:00,188.734,189.508,188.734,189.354,189.370,189.339,1,True,188.620034,72.444544,1,190.2840,188.734
384,2024-08-14 07:00:00+00:00,188.918,189.278,188.864,189.255,189.268,189.242,1,True,188.672254,58.523445,1,189.7605,188.918
399,2024-08-14 22:00:00+00:00,188.926,188.970,188.768,188.943,188.961,188.925,1,True,188.699381,52.452323,1,188.9685,188.926
402,2024-08-15 01:00:00+00:00,188.854,189.319,188.803,189.236,189.252,189.221,1,True,188.708007,58.707240,1,189.8090,188.854
407,2024-08-15 06:00:00+00:00,188.977,189.293,188.928,189.150,189.166,189.133,1,True,188.728002,55.591381,1,189.4095,188.977


In [53]:
df_plot = df_slim.iloc[2068:2080]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[cp.df_plot.SIGNAL != NONE]
markers = ['mid_c', 'TP', 'SL']
marker_colors = ["#0000FF", "#00FF00", "#FF0000"]
for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x = trades.sTime,
        y = trades[markers[i]],
        mode = 'markers',
        marker = dict(color=marker_colors[i], size=12)
    ))
cp.show_plot(line_traces=["EMA_200"], sec_traces=['RSI_14'])

In [ ]:
open_trades = []
for index, row in df_slim.iterrows():
    

In [54]:
df_raw.arl()

AttributeError: 'DataFrame' object has no attribute 'arl'